In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# 학습 데이터 로드 및 전처리
train_data = pd.read_csv('./gist-mldl24f-hw3/train.csv')

# 날짜와 시간 데이터 전처리
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['num_code'] = train_data['num_code'].astype(str)

# 요일 반환 함수 정의
def get_day_of_week(date):
    try:
        return date.day_name()
    except AttributeError:
        return np.nan
    
# 요일 데이터 칼럼 추가
train_data['day_of_week'] = train_data['date'].apply(get_day_of_week)

# 시간 변환 함수 정의
def convert_time_to_minutes(time_str):
    try:
        hours, minutes = time_str.split('h')
        hours = float(hours.strip())
        minutes = int(minutes.replace('m', '').strip())
        total_minutes = int(hours * 60) + minutes
        return total_minutes
    except ValueError:
        return np.nan

train_data['time_taken'] = train_data['time_taken'].apply(convert_time_to_minutes)

# 출발 및 도착 시간 전처리
train_data['dep_time'] = pd.to_datetime(train_data['dep_time'], format='%H:%M')
train_data['arr_time'] = pd.to_datetime(train_data['arr_time'], format='%H:%M')

train_data['dep_hour'] = train_data['dep_time'].dt.hour
train_data['arr_hour'] = train_data['arr_time'].dt.hour

# 시간대를 범주형으로 추가
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

train_data['dep_time_period'] = train_data['dep_time'].dt.hour.apply(time_of_day)
train_data['arr_time_period'] = train_data['arr_time'].dt.hour.apply(time_of_day)

# stop 정보를 숫자로 변환
train_data['stop_num'] = train_data['stop'].map({
    'non-stop': 0,
    '1-stop': 1,
    '2+-stop': 2
})

# 기준 날짜로부터의 경과 일수 추가
reference_date = pd.to_datetime('2022-02-11')
train_data['days_since'] = (pd.to_datetime(train_data['date']) - reference_date).dt.days

# One-Hot Encoding을 위한 범주형 변수 목록
cat_predictors = ['airline', 'from', 'to', 'class', 'dep_time_period', 'arr_time_period', 'day_of_week']
num_predictors = ['time_taken', 'days_since', 'dep_hour', 'arr_hour', 'stop_num']

# 범주형 변수에 대해 One-Hot Encoding 적용 및 수치형 변수와 결합
X_cat_train = pd.get_dummies(train_data[cat_predictors], drop_first=True)
X_num_train = train_data[num_predictors]
X_train = pd.concat([X_num_train, X_cat_train], axis=1)
y_train = train_data['price']

# XGBoost 모델 생성 및 학습
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=0)
xgb_model.fit(X_train, y_train)

# 테스트 데이터 로드 및 동일한 전처리 적용
test_data = pd.read_csv('./gist-mldl24f-hw3/test.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data['time_taken'] = test_data['time_taken'].apply(convert_time_to_minutes)
test_data['dep_time'] = pd.to_datetime(test_data['dep_time'], format='%H:%M')
test_data['arr_time'] = pd.to_datetime(test_data['arr_time'], format='%H:%M')
test_data['dep_time_period'] = test_data['dep_time'].dt.hour.apply(time_of_day)
test_data['arr_time_period'] = test_data['arr_time'].dt.hour.apply(time_of_day)
test_data['stop_num'] = test_data['stop'].map({'non-stop': 0, '1-stop': 1, '2+-stop': 2})
test_data['days_since'] = (pd.to_datetime(test_data['date']) - reference_date).dt.days
test_data['dep_hour'] = test_data['dep_time'].dt.hour
test_data['arr_hour'] = test_data['arr_time'].dt.hour
test_data['stop_num'] = test_data['stop'].map({
    'non-stop': 0,
    '1-stop': 1,
    '2+-stop': 2
})
test_data['day_of_week'] = test_data['date'].apply(get_day_of_week)

# 테스트 데이터에 One-Hot Encoding 적용 및 학습 데이터와 동일한 열 맞추기
X_cat_test = pd.get_dummies(test_data[cat_predictors], drop_first=True)
X_num_test = test_data[num_predictors]
X_test = pd.concat([X_num_test, X_cat_test], axis=1)

# 학습 데이터와 테스트 데이터 열 일치
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 결과 DataFrame 생성
result = pd.DataFrame({'id': test_data['id'], '4_pred': y_pred_ensemble})

# CSV 파일로 저장
result.to_csv('5_pred.csv', index=False)
print("5_pred.csv 파일이 생성되었습니다.")


4_pred.csv 파일이 생성되었습니다.
